In [ ]:
import pandas as pd
import numpy as np
import os

## Data Loading

In [ ]:
source_data = pd.read_csv("/Users/alexandreheringer/31-Other-Projects/Research/12 - Inputs/sample_data.csv")

print(source_data.head())
print(source_data.info())

## Technology Classification

In [ ]:
technology_classification_conditions = [
    (source_data['technology'] == 10) & (source_data['max_advertised_download_speed'] < 50),
    (source_data['technology'] == 10) & (source_data['max_advertised_download_speed'] >= 50),
    (source_data['technology'] == 40) & (source_data['max_advertised_download_speed'] < 500),
    (source_data['technology'] == 40) & (source_data['max_advertised_download_speed'] >= 500),
    (source_data['technology'] == 50)
]

technology_classification_labels = [
    'Slow Copper',
    'Fast Copper',
    'Slow Cable',
    'Fast Cable',
    'Fiber'
]

source_data['technology_category'] = np.select(
    technology_classification_conditions,
    technology_classification_labels,
    default='Other'
)

print(source_data.head())

## Gigabit-Capable Provider Identification

In [ ]:
#Create function to check if it's gigabit:

# If fiber then it's gigabit
# If technology = 40 & download_speed >= 500
# Vectorized implementation (replaces row-by-row apply for performance)
source_data['is_gigabit'] = (
    (source_data['technology'] == 50)
    | (
        (source_data['technology'] == 40)
        & (source_data['max_advertised_download_speed'] >= 500)
    )
).astype(int)

# create check coulmn and apply function for each row
# vectorized above instead of using apply()

# Note: competition-related columns (current_gigabit_providers, available_spots,
# competition_score) were previously computed here but are NOT needed by the
# simulation — it recalculates everything from scratch each year.
# They are now computed only in the Exploratory section below where they are used.

print(source_data.head())

## Identify Eligible Contenders

In [ ]:
#1. UPGRADES: Providers already at the location with slow technology

# Filter rows eligible for upgrade
# Vectorized implementation
upgrade_opportunities = source_data[
    source_data['technology_category'].isin(['Slow Copper', 'Slow Cable'])
].copy()
upgrade_opportunities['growth_type'] = 'Upgrade'

# Keep essential columns for the build queue
upgrade_opportunities = upgrade_opportunities[
    ['location_id', 'block_geoid', 'provider_id', 'growth_type']
]

In [ ]:
#2. EDGE-OUT: Providers in the census block but not at this specific location

# 1. Identify all unique providers present in each block
providers_per_block = source_data[['block_geoid', 'provider_id']].drop_duplicates()

# 2. Identify all unique locations in each block
locations_per_block = source_data[
    ['location_id', 'block_geoid']
].drop_duplicates()

# 3. Cartesian Product (within blocks): Every provider in the block mapped to every location in that block
potential_edge_out_pairs = locations_per_block.merge(
    providers_per_block, on='block_geoid'
)

# 4. Filter out combinations where the provider is already physically present at the location
existing_presence = source_data[['location_id', 'provider_id']].assign(is_present=True)
edge_out_opportunities = potential_edge_out_pairs.merge(
    existing_presence, on=['location_id', 'provider_id'], how='left'
)

# Keep only the 'NaN' results (where is_present is null)
edge_out_opportunities = edge_out_opportunities[
    edge_out_opportunities['is_present'].isna()
].copy()
edge_out_opportunities['growth_type'] = 'Edge-Out'
edge_out_opportunities = edge_out_opportunities.drop(columns=['is_present'])

In [ ]:
#3.  Unify All Contenders ---
# Combine Upgrade and Edge-Out candidates into a single master table for scoring
all_contenders = pd.concat(
    [upgrade_opportunities, edge_out_opportunities], ignore_index=True
)

print(f'Total Upgrade opportunities found: {len(upgrade_opportunities)}')
print(f'Total Edge-Out opportunities found: {len(edge_out_opportunities)}')
print(all_contenders.head())

## Exploratory: Pre-Simulation Scoring Verification

> **Note:** This section computes the composite score on the initial contenders dataset
> for verification and exploration purposes only. The actual multi-year simulation below
> recalculates all scores from scratch each year based on the updated market state.
> This section can be safely skipped without affecting the simulation results.

In [ ]:
#1 Calculate Block Market Size
# Count how many unique locations are in each block
block_location_counts = (
    source_data.groupby('block_geoid')['location_id'].nunique().reset_index()
)
block_location_counts.columns = ['block_geoid', 'location_count']

In [ ]:
#2: Calculation Provider Gigabit Coverage per Block
# Count how many gigabit locations a provider already has in each block
gigabit_counts_per_block = (
    source_data.groupby(['provider_id', 'block_geoid'])['is_gigabit']
    .sum()
    .reset_index()
)
gigabit_counts_per_block.columns = [
    'provider_id', 'block_geoid', 'gigabit_location_count'
]

In [ ]:
#3: Combine scores into the contenders table
all_contenders = all_contenders.merge(
    block_location_counts, on='block_geoid', how='left'
)
all_contenders = all_contenders.merge(
    gigabit_counts_per_block, on=['provider_id', 'block_geoid'], how='left'
).fillna(0)

# Competition score (computed here for exploratory purposes only;
# the simulation recalculates this from scratch each year)
# counts and apply results to respective correct line
exploratory_gigabit_counts = (
    source_data.groupby('location_id')['is_gigabit'].sum().reset_index()
)
exploratory_gigabit_counts.columns = ['location_id', 'current_gigabit_providers']
all_contenders = all_contenders.merge(
    exploratory_gigabit_counts, on='location_id', how='left'
)
all_contenders['current_gigabit_providers'] = (
    all_contenders['current_gigabit_providers'].fillna(0)
)
# Calculate and the spot availability
all_contenders['available_spots'] = 2 - all_contenders['current_gigabit_providers']
# if value is negative then == 0(grandfathered)
all_contenders.loc[all_contenders['available_spots'] < 0, 'available_spots'] = 0
#Create competition score normalizing available spots
all_contenders['competition_score'] = all_contenders['available_spots'] / 2

In [ ]:
#4: Normalize scores to 0-1 scale
# Normalize Market Size
min_size = all_contenders['location_count'].min()
max_size = all_contenders['location_count'].max()
all_contenders['market_size_score'] = (
    (all_contenders['location_count'] - min_size) / (max_size - min_size)
)

# Calculate and Normalize Coverage Ratio
all_contenders['coverage_ratio'] = (
    all_contenders['gigabit_location_count'] / all_contenders['location_count']
)
min_coverage = all_contenders['coverage_ratio'].min()
max_coverage = all_contenders['coverage_ratio'].max()
all_contenders['gigabit_coverage_score'] = (
    (all_contenders['coverage_ratio'] - min_coverage)
    / (max_coverage - min_coverage)
)

In [ ]:
# Final Composite Score: Average of the three components
all_contenders['composite_score'] = (
    all_contenders['competition_score']
    + all_contenders['gigabit_coverage_score']
    + all_contenders['market_size_score']
) / 3

# Sort by priority (Highest score first)
all_contenders = all_contenders.sort_values(
    by='composite_score', ascending=False
)

print('Top prioritized build opportunities (exploratory):')
print(
    all_contenders[
        ['location_id', 'provider_id', 'growth_type', 'composite_score']
    ].head()
)

## MULTI-YEAR SIMULATION

In [ ]:
#1: Simulation setup
#Creates a database to store the market state over the years
market_state = source_data.copy()

#creates the log of all construction events
build_history = []

#calculates the state annual building capacity 
total_unique_locations = market_state['location_id'].nunique()
state_annual_capacity = int(0.02 * total_unique_locations)

#2: Calculation of all possible Edge-Outs
# If a provider is on the block, he will always be a possible Edge-Out
# What changes over years is if he already built (then exit the list) or not.

providers_per_block_static = (
    market_state[['block_geoid', 'provider_id']].drop_duplicates()
)
locations_per_block_static = (
    market_state[['location_id', 'block_geoid']].drop_duplicates()
)

#Total universe of edge outs
edge_out_universe = locations_per_block_static.merge(
    providers_per_block_static, on='block_geoid'
)
edge_out_universe['growth_type'] = 'Edge-Out'

# Block location counts dictionary (static throughout the simulation)
block_location_counts_dict = (
    market_state.groupby('block_geoid')['location_id'].nunique().to_dict()
)

print(f'Total unique locations: {total_unique_locations}')
print(f'State annual capacity: {state_annual_capacity} builds/year')

In [ ]:
#3: Simulation Loop
for year in range(2026, 2036):
    print(f"\n{'=' * 60}")
    print(f">>> Simulating Year: {year}")
    print(f"{'=' * 60}")

    #3.1 - Establish Capacities

    #Counts how many unique locations each provider serves acctualy.
    provider_footprints = market_state.groupby('provider_id')['location_id'].nunique()

    #Variable provider_caps -> increases over time,
    #current_state_remaining -> gets reseted to the same fixed level.
    provider_capacities = (provider_footprints * 0.04).astype(int).to_dict()
    state_remaining_capacity = state_annual_capacity

    #3.2: Fast Trackers:
    #Creates a dictionary with location id and how many Gigabits providers exists on it.
    #used to check the rule of limit of 2 providers per location.
    location_gigabit_counts = (
        market_state.groupby('location_id')['is_gigabit'].sum().to_dict()
    )

    #Used to calculate Coverage Score.
    # Understands if the provider already is strong in the block, in order to prioritize it.
    provider_gigabit_by_block = (
        market_state.groupby(['provider_id', 'block_geoid'])['is_gigabit']
        .sum()
        .reset_index()
        .rename(columns={'is_gigabit': 'provider_gigabit_count_in_block'})
    )

    # Used to rank contenders within each location (tie-breaking)
    provider_total_locations_by_block = (
        market_state.groupby(['provider_id', 'block_geoid'])['location_id']
        .nunique()
        .reset_index()
        .rename(columns={'location_id': 'provider_total_locations_in_block'})
    )

    #3.3: Identify Contenders
    #A. UPGRADES -  Identify who should upgrade each year. Changes YoY.
    upgrade_candidates = market_state[
        market_state['technology_category'].isin(['Slow Copper', 'Slow Cable'])
    ][['location_id', 'block_geoid', 'provider_id']].copy()
    upgrade_candidates['growth_type'] = 'Upgrade'

    #B. EDGE-OUTS
    # Creates a set of al the locations,providers on the actual market.

    #Instead of using this function below, I've decided to use a vector approach in order to save memory
    #def filter_existing(row):
    #    return (row['location_id'], row['provider_id']) not in existing_pairs

    #Select universe, select existing locations and providers and creates a mask with the subtraction
    #generating mask_not_exists variable.
    universe_index = pd.MultiIndex.from_frame(
        edge_out_universe[['location_id', 'provider_id']]
    )
    existing_index = pd.MultiIndex.from_frame(
        market_state[['location_id', 'provider_id']]
    )
    not_yet_present_mask = ~universe_index.isin(existing_index)

    #creates a valid list of edge out expansions for this year.
    edge_out_candidates = edge_out_universe[not_yet_present_mask].copy()

    #unify upgrades and edge out cases for this year.
    candidates = pd.concat(
        [upgrade_candidates, edge_out_candidates], ignore_index=True
    )

    if candidates.empty:
        print("No more build opportunities. Stopping simulation.")
        break

    #3.4:Update competition score to check who has priority.
    candidates['current_gigabit_count'] = (
        candidates['location_id'].map(location_gigabit_counts).fillna(0)
    )
    candidates['competition_score'] = (
        (2 - candidates['current_gigabit_count']).clip(lower=0) / 2
    )

    #insert market size into candidates df and normalize market score.
    candidates['block_location_count'] = candidates['block_geoid'].map(
        block_location_counts_dict
    )
    min_size = candidates['block_location_count'].min()
    max_size = candidates['block_location_count'].max()
    candidates['market_size_score'] = (
        (candidates['block_location_count'] - min_size)
        / (max_size - min_size + 1e-9)
    )

    # Calculates the Coverage Score (location occupation by provider) and normalize it.
    # --- R2 Performance: replaced .apply(axis=1) with merge ---
    # Why: .apply() runs a Python-level loop over every row (~138k candidates/year).
    # merge() delegates the join to pandas' optimized C internals, which is
    # significantly faster. The result is identical: each candidate row gets the
    # provider's gigabit count in its block, looked up by (provider_id, block_geoid).
    candidates = candidates.merge(
        provider_gigabit_by_block,
        on=['provider_id', 'block_geoid'],
        how='left',
    )
    candidates['provider_gigabit_count_in_block'] = (
        candidates['provider_gigabit_count_in_block'].fillna(0)
    )

    candidates['coverage_ratio'] = (
        candidates['provider_gigabit_count_in_block']
        / candidates['block_location_count']
    )
    min_coverage = candidates['coverage_ratio'].min()
    max_coverage = candidates['coverage_ratio'].max()
    candidates['gigabit_coverage_score'] = (
        (candidates['coverage_ratio'] - min_coverage)
        / (max_coverage - min_coverage + 1e-9)
    )

    #Final Ranking (average between the 3 scores).
    candidates['composite_score'] = (
        candidates['competition_score']
        + candidates['gigabit_coverage_score']
        + candidates['market_size_score']
    ) / 3

    # Tie-breaking columns for within-location contender prioritization.
    # --- R2 Performance: replaced .apply(axis=1) with merge (same rationale as above) ---
    candidates = candidates.merge(
        provider_total_locations_by_block,
        on=['provider_id', 'block_geoid'],
        how='left',
    )
    candidates['provider_total_locations_in_block'] = (
        candidates['provider_total_locations_in_block'].fillna(0)
    )

    candidates['type_priority'] = candidates['growth_type'].map(
        {'Upgrade': 0, 'Edge-Out': 1}
    )

    #3.5 Allocation Process (Greedy Algorithm)
    # --- R3 Performance: Pre-group candidates by location ---
    # Why: Previously, the loop filtered `candidates[candidates['location_id'] == X]`
    # for every location, scanning the full DataFrame each time (O(locations × candidates)).
    # By grouping once upfront and pre-sorting each group, the allocation loop
    # simply iterates through pre-built lists — no repeated filtering or sorting.
    candidates_by_location = {}
    for loc_id, group in candidates.groupby('location_id'):
        candidates_by_location[loc_id] = group.sort_values(
            by=[
                'type_priority',
                'provider_gigabit_count_in_block',
                'provider_total_locations_in_block',
            ],
            ascending=[True, False, False],
        )

    # Determine location processing order by maximum composite score at each location
    location_processing_order = (
        candidates.groupby('location_id')['composite_score']
        .max()
        .sort_values(ascending=False)
        .index
    )

    year_build_log = []
    new_edge_out_rows = []  # List to accumulate new records for bulk insertion

    # Iterate through candidates ranked by priority (Highest Score -> Lowest Score)
    # Now iterating location-by-location, trying all contenders per location before moving on
    for current_location_id in location_processing_order:
        # Check State Constraint: Stop entire year if state cap is hit
        if state_remaining_capacity <= 0:
            break

        # Check Saturation Constraint: Skip if location already has 2+ Gigabit providers
        if location_gigabit_counts.get(current_location_id, 0) >= 2:
            continue

        # Get contenders for this location (pre-sorted by priority):
        # 1. Upgrades before edge-outs (type_priority ascending)
        # 2. Higher gigabit count in block (descending)
        # 3. Higher total locations in block (descending)
        location_contenders = candidates_by_location[current_location_id]

        for _, contender in location_contenders.iterrows():
            current_provider_id = contender['provider_id']

            # Check Provider Constraint: Skip if provider hit their 4% annual limit.
            if provider_capacities.get(current_provider_id, 0) <= 0:
                continue

            # Build Confirmed: Record the transaction
            year_build_log.append({
                'year': year,
                'provider_id': current_provider_id,
                'location_id': current_location_id,
                'growth_type': contender['growth_type'],
            })

            # Update Real-Time Constraints (decrement caps, increment saturation).
            state_remaining_capacity -= 1
            provider_capacities[current_provider_id] -= 1
            location_gigabit_counts[current_location_id] = (
                location_gigabit_counts.get(current_location_id, 0) + 1
            )

            # Update Market State (for next year's logic).
            if contender['growth_type'] == 'Upgrade':
                # For Upgrades: Locate existing row and update technology to Fiber.
                upgrade_mask = (
                    (market_state['location_id'] == current_location_id)
                    & (market_state['provider_id'] == current_provider_id)
                )
                market_state.loc[
                    upgrade_mask,
                    ['technology', 'technology_category', 'is_gigabit'],
                ] = [50, 'Fiber', 1]
            else:
                # For Edge-Outs: Stage new service row for bulk insertion.
                new_edge_out_rows.append({
                    'location_id': current_location_id,
                    'block_geoid': contender['block_geoid'],
                    'provider_id': current_provider_id,
                    'technology': 50,
                    'max_advertised_download_speed': 1000,
                    'max_advertised_upload_speed': 1000,
                    'technology_category': 'Fiber',
                    'is_gigabit': 1,
                })

            # One build per available spot; move to next location.
            break

    #Bulk insert all new Edge-Out rows at once.
    if new_edge_out_rows:
        market_state = pd.concat(
            [market_state, pd.DataFrame(new_edge_out_rows)], ignore_index=True
        )

    build_history.extend(year_build_log)

    upgrade_count = sum(
        1 for build in year_build_log if build['growth_type'] == 'Upgrade'
    )
    edge_out_count = sum(
        1 for build in year_build_log if build['growth_type'] == 'Edge-Out'
    )
    print(
        f"Year {year} finished: {len(year_build_log)} works carried out. "
        f"({upgrade_count} upgrades, {edge_out_count} edge-outs)"
    )

    if len(year_build_log) == 0:
        print("No builds possible this year. Stopping simulation.")
        break

## Final Report

In [ ]:
if build_history:
    # Convert the list of build dictionaries into a DataFrame.
    build_history_dataframe = pd.DataFrame(build_history)

    # Aggregate builds by year, provider, and type.
    # 'unstack' pivots the growth_type to columns (Upgrade, Edge-Out).
    summary = (
        build_history_dataframe
        .groupby(['year', 'provider_id', 'growth_type'])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )

    # Rename columns to match the required output format.
    summary = summary.rename(columns={
        'Upgrade': 'upgrade_builds',
        'Edge-Out': 'edge_out_builds'
    })

    # Ensure both column types exist (even if 0 builds of that type occurred).
    for column_name in ['upgrade_builds', 'edge_out_builds']:
        if column_name not in summary.columns:
            summary[column_name] = 0

    # Calculate total builds per provider per year.
    summary['total_builds'] = (
        summary['upgrade_builds'] + summary['edge_out_builds']
    )

    # Sort by year (ascending) and total builds (descending).
    print('\n--- Year-Over-Year Summary Table ---')
    print(
        summary.sort_values(
            ['year', 'total_builds'], ascending=[True, False]
        ).to_string()
    )
else:
    print('No builds were recorded during the simulation.')

In [ ]:
# Export summary table to CSV.
output_directory = (
    "/Users/alexandreheringer/31-Other-Projects/Research/14 - Outputs"
)
os.makedirs(output_directory, exist_ok=True)

output_path = os.path.join(output_directory, 'yoy_summary.csv')
summary.to_csv(output_path, index=False)
print(f'Summary table exported to: {output_path}')

## Analysis Report

### Which providers are growing the fastest and why?

PROV_005 leads with 1,039 total builds over 10 years, followed by PROV_001 (992), PROV_007 (979), PROV_008 (962), and PROV_011 (937). These providers grow fastest because the 4% annual capacity rule scales directly with footprint size — PROV_005 starts with the largest footprint (capacity of 358 builds in Year 1) and each year's builds expand its footprint further, creating a compounding advantage. Conversely, smaller-footprint providers like PROV_010 (509 total) and PROV_012 (474 total) consistently receive fewer build allocations because their 4% cap is smaller.

### Which growth type (upgrades vs. edge-out) dominates in early years vs. later years?

Upgrades dominate exclusively in Year 1 (all 1,219 builds are upgrades, 0 edge-outs), since the specification gives upgrades strict priority over edge-outs and there are ~12,163 initial upgrade opportunities. Edge-outs first overtake upgrades in Year 2 (741 edge-outs vs. 478 upgrades). By Year 5 (2030), the split is 1,029 edge-outs vs. 190 upgrades. Upgrades never fully exhaust during the 10-year simulation (234 remain in Year 10), because some upgrade opportunities are at lower-scored locations that are continually outranked by higher-scored edge-out candidates.

### What happens to the state-level and provider-level capacity constraints over time?

The state-level capacity (1,219 builds/year = 2% of 60,961 locations) is the binding constraint in every single year of the simulation — all 10 years hit exactly 1,219 builds. Provider-level capacities, starting at ~292–358 builds/year, grow via compounding as each year's builds expand footprints. Their aggregate sum far exceeds the state ceiling, meaning individual provider caps are never the bottleneck at the system level, though they do shape which providers receive allocations in any given year.